In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from matplotlib.ticker import PercentFormatter
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import skmob
from skmob.preprocessing import detection
from skmob.tessellation import tilers
from skmob.measures.collective import visits_per_location
from skmob.measures.individual import distance_straight_line, home_location, maximum_distance, number_of_locations, location_frequency

In [142]:
ventas_client_gye = pd.read_csv("../../data/output/dfgeo_region_2_concat.zip", compression="zip")


/tmp/ipykernel_62717/4151638997.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  ventas_client_gye = pd.read_csv("../../data/output/dfgeo_region_2_concat.zip", compression="zip")


In [143]:
ventas_client_gye.head()

,fecha_ingreso,grupo,cadena,comercio,estado,pais,region,provincia,ciudad,zona,...,Name,Description,fecha,hora,proveedor,producto,medio,total,cliente_final,saldo_despues
0,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,NaN,2019-01-17,18:24:06,5.0,2.0,78,1.0,%..)!!.!*',8.93
1,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,NaN,2019-01-25,10:12:50,83.0,2.0,78,3.0,%.!---'&.-,9.18
2,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,NaN,2019-01-25,10:11:45,83.0,2.0,78,10.0,%.!-!-&.'%,18.73
3,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,NaN,2019-01-21,18:11:47,5.0,15.0,78,1.0,%.-%)+%)-%,37.58
4,2019-01-03,4,4,90093,0,1.0,2.0,4.0,24.0,26.0,...,Rosales,NaN,2019-01-12,10:02:09,5.0,15.0,78,3.0,%..'%((*%+,34.57


In [144]:
ventas_client_gye.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520081 entries, 0 to 520080
Data columns (total 30 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fecha_ingreso   520081 non-null  object 
 1   grupo           520081 non-null  int64  
 2   cadena          520081 non-null  int64  
 3   comercio        520081 non-null  int64  
 4   estado          520081 non-null  object 
 5   pais            520081 non-null  float64
 6   region          520081 non-null  float64
 7   provincia       520081 non-null  float64
 8   ciudad          520081 non-null  float64
 9   zona            520081 non-null  float64
 10  ejecutivo       520081 non-null  float64
 11  latitud         520081 non-null  float64
 12  longitud        520081 non-null  float64
 13  geometry        520081 non-null  object 
 14  index_right     520081 non-null  int64  
 15  stroke          519220 non-null  object 
 16  stroke-width    519220 non-null  float64
 17  stroke-opa

In [145]:
ventas_client_gye.shape

(520081, 30)

In [146]:
ventas_client_gye.isna().sum()

fecha_ingreso          0
grupo                  0
cadena                 0
comercio               0
estado                 0
pais                   0
region                 0
provincia              0
ciudad                 0
zona                   0
ejecutivo              0
latitud                0
longitud               0
geometry               0
index_right            0
stroke               861
stroke-width         861
stroke-opacity       861
fill                 861
fill-opacity         861
Name                   0
Description       517359
fecha                  0
hora                   0
proveedor              0
producto               0
medio                  0
total                  1
cliente_final          0
saldo_despues          0
dtype: int64

In [147]:
columns_manipulation = ['fecha', 'hora', 'Name','comercio', 'proveedor', 'producto','ejecutivo','latitud', 'longitud', 'cliente_final']

In [148]:
ventas_client_gye_manipulation = ventas_client_gye.loc[:, columns_manipulation]

In [149]:
ventas_client_gye_manipulation.head()

,fecha,hora,Name,comercio,proveedor,producto,ejecutivo,latitud,longitud,cliente_final
0,2019-01-17,18:24:06,Rosales,90093,5.0,2.0,367.0,-2.115323,-79.910043,%..)!!.!*'
1,2019-01-25,10:12:50,Rosales,90093,83.0,2.0,367.0,-2.115323,-79.910043,%.!---'&.-
2,2019-01-25,10:11:45,Rosales,90093,83.0,2.0,367.0,-2.115323,-79.910043,%.!-!-&.'%
3,2019-01-21,18:11:47,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%
4,2019-01-12,10:02:09,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%..'%((*%+


In [150]:
columns_use = ['fecha', 'latitud', 'longitud', 'cliente_final']

In [151]:
idx = pd.IndexSlice
ventas_client_gye = ventas_client_gye.loc[idx[:, columns_use]]
ventas_client_gye

,fecha,latitud,longitud,cliente_final
0,2019-01-17,-2.115323,-79.910043,%..)!!.!*'
1,2019-01-25,-2.115323,-79.910043,%.!---'&.-
2,2019-01-25,-2.115323,-79.910043,%.!-!-&.'%
3,2019-01-21,-2.115323,-79.910043,%.-%)+%)-%
4,2019-01-12,-2.115323,-79.910043,%..'%((*%+
...,...,...,...,...
520076,2019-12-10,-2.081278,-79.897726,'%&%%&%--!-)
520077,2019-12-14,-2.081278,-79.897726,%.(.)&*+%%
520078,2019-12-16,-2.081278,-79.897726,%..%.'%%!%
520079,2019-12-02,-2.081278,-79.897726,%.+-+-'**-


In [152]:
ventas_client_gye.head()

,fecha,latitud,longitud,cliente_final
0,2019-01-17,-2.115323,-79.910043,%..)!!.!*'
1,2019-01-25,-2.115323,-79.910043,%.!---'&.-
2,2019-01-25,-2.115323,-79.910043,%.!-!-&.'%
3,2019-01-21,-2.115323,-79.910043,%.-%)+%)-%
4,2019-01-12,-2.115323,-79.910043,%..'%((*%+


In [153]:
ventas_client_gye['cliente_final'].value_counts()

%.+!%+&'*'    278
%.+!!&&.!.    235
%.--&*+)'.    195
%..)&*!&-!    191
%...('!*''    190
             ... 
%.+&*&%(-%      1
%.+.-)+*+&      1
%.+-'!.-!'      1
%.(.%+!*)%      1
%)(--(-..       1
Name: cliente_final, Length: 216911, dtype: int64

In [154]:
ventas_client_gye.shape

(520081, 4)

In [155]:

ventas_client_gye_user_have_more_than_six_record = ventas_client_gye_manipulation.groupby('cliente_final').filter(lambda x: len(x) > 6)

In [156]:
ventas_client_gye_user_have_more_than_six_record.shape

(197258, 10)

In [157]:
ventas_client_gye_user_have_more_than_six_record.head()

,fecha,hora,Name,comercio,proveedor,producto,ejecutivo,latitud,longitud,cliente_final
3,2019-01-21,18:11:47,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%
15,2019-01-03,11:40:27,Rosales,90093,5.0,2.0,367.0,-2.115323,-79.910043,%.+-'%*%!)
16,2019-01-12,14:32:17,Rosales,90093,5.0,2.0,367.0,-2.115323,-79.910043,%..!(-+.*-
23,2019-01-20,17:19:18,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%
26,2019-01-12,18:42:45,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%


In [158]:
ventas_client_gye_user_have_more_than_six_record['cliente_final'].value_counts()

%.+!%+&'*'      278
%.+!!&&.!.      235
%.--&*+)'.      195
%..)&*!&-!      191
%...('!*''      190
               ... 
%.+%.&&'%)        7
%.-&&'%*!*        7
%..!&&)!%!        7
%.-'*%'!!!        7
%%%!*+!+.&*)      7
Name: cliente_final, Length: 13278, dtype: int64

In [159]:
tdf = skmob.TrajDataFrame(ventas_client_gye_user_have_more_than_six_record, latitude='latitud', longitude='longitud', datetime='fecha', user_id='cliente_final')
tdf.head()

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid
3,2019-01-21,18:11:47,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%
15,2019-01-03,11:40:27,Rosales,90093,5.0,2.0,367.0,-2.115323,-79.910043,%.+-'%*%!)
16,2019-01-12,14:32:17,Rosales,90093,5.0,2.0,367.0,-2.115323,-79.910043,%..!(-+.*-
23,2019-01-20,17:19:18,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%
26,2019-01-12,18:42:45,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%


In [160]:
tdf.shape

(197258, 10)

### CVS to user have mobility data

In [161]:
dsl_df = distance_straight_line(tdf)
dsl_df.head()

100%|██████████| 13278/13278 [00:08<00:00, 1528.69it/s]


,uid,distance_straight_line
0,!!(&&-*,0.0
1,!'+!&&,0.0
2,!'+!&*,0.0
3,!'++(',0.0
4,%%%!!&(!&)&+,0.0


In [162]:
dsl_df['distance_straight_line'].max()

950.0536433396779

In [163]:
nl_df = number_of_locations(tdf)

100%|██████████| 13278/13278 [00:09<00:00, 1458.22it/s]


In [164]:
nl_df['number_of_locations'].value_counts()

1     8822
2     3181
3      895
4      265
5       70
6       27
7       12
10       2
9        2
20       1
8        1
Name: number_of_locations, dtype: int64

In [165]:
nl_df.tail()

,uid,number_of_locations
13273,-*((++,2
13274,.!(*,1
13275,.&'.-(,1
13276,.)%%%-*),1
13277,cVc,10


In [166]:
tdf_n_visits_distance_straight_line = tdf.merge(dsl_df, on='uid')
tdf_n_visits_distance_straight_line

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid,distance_straight_line
0,2019-01-21,18:11:47,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0
1,2019-01-20,17:19:18,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0
2,2019-01-12,18:42:45,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0
3,2019-01-14,15:53:18,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0
4,2019-01-10,13:28:34,Rosales,90093,5.0,2.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0
...,...,...,...,...,...,...,...,...,...,...,...
197253,2019-12-24,19:22:58,Geranios 1,95834,5.0,15.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0
197254,2019-12-31,10:16:25,Geranios 1,95834,5.0,2.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0
197255,2019-12-16,18:52:04,Geranios 1,95834,5.0,15.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0
197256,2019-12-30,19:14:03,Geranios 1,95834,5.0,15.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0


In [167]:
tdf_n_visits_distance_straight_line_n_locations = tdf_n_visits_distance_straight_line.merge(nl_df, on='uid')
tdf_n_visits_distance_straight_line_n_locations

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid,distance_straight_line,number_of_locations
0,2019-01-21,18:11:47,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0,1
1,2019-01-20,17:19:18,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0,1
2,2019-01-12,18:42:45,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0,1
3,2019-01-14,15:53:18,Rosales,90093,5.0,15.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0,1
4,2019-01-10,13:28:34,Rosales,90093,5.0,2.0,367.0,-2.115323,-79.910043,%.-%)+%)-%,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
197253,2019-12-24,19:22:58,Geranios 1,95834,5.0,15.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0,1
197254,2019-12-31,10:16:25,Geranios 1,95834,5.0,2.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0,1
197255,2019-12-16,18:52:04,Geranios 1,95834,5.0,15.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0,1
197256,2019-12-30,19:14:03,Geranios 1,95834,5.0,15.0,367.0,-2.081278,-79.897726,%..!!!)*--,0.0,1


In [168]:
df_users_have_more_than_1_location = tdf_n_visits_distance_straight_line_n_locations.loc[tdf_n_visits_distance_straight_line_n_locations['number_of_locations'] > 1]

In [169]:
df_users_have_more_than_1_location.head()

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid,distance_straight_line,number_of_locations
105,2019-02-21,09:10:48,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3
106,2019-02-07,09:12:07,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3
107,2019-04-16,21:30:15,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3
108,2019-04-09,20:35:36,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3
109,2019-04-22,17:28:48,Samanes,92593,6.0,2.0,367.0,-2.119984,-79.904241,%.+%+-')%!,4.754176,3


In [170]:
df_users_have_more_than_1_location['number_of_locations'].value_counts()

2     48527
3     15452
4      4512
5      1004
6       523
20      184
7       168
9        25
10       24
8        13
Name: number_of_locations, dtype: int64

In [171]:
df_users_have_more_than_1_location['uid'].value_counts()

%.+!%+&'*'    278
%.+!!&&.!.    235
%..)&*!&-!    191
%...('!*''    190
%%(           184
             ... 
%.+&('++)-      7
%.!.&((+')      7
%..')!+.))      7
%.+-%.(*&.      7
%..&'+--)&      7
Name: uid, Length: 4456, dtype: int64

In [172]:
df_users_have_more_than_1_location.shape

(70432, 12)

In [173]:
tdf_mobility = skmob.TrajDataFrame(df_users_have_more_than_1_location, latitude='latitud', longitude='longitud', datetime='fecha', user_id='cliente_final')
tdf_mobility.head()

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid,distance_straight_line,number_of_locations
105,2019-02-21,09:10:48,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3
106,2019-02-07,09:12:07,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3
107,2019-04-16,21:30:15,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3
108,2019-04-09,20:35:36,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3
109,2019-04-22,17:28:48,Samanes,92593,6.0,2.0,367.0,-2.119984,-79.904241,%.+%+-')%!,4.754176,3


In [174]:
vl_df = visits_per_location(tdf_mobility)
vl_df

,lat,lng,n_visits
0,-2.202628,-79.926652,7029
1,-2.102911,-79.949303,4097
2,-2.202541,-79.926314,3758
3,-2.103280,-79.948524,3326
4,-2.232126,-79.906007,2504
...,...,...,...
289,-2.157958,-79.911498,1
290,-2.206502,-79.907119,1
291,-2.099081,-79.903293,1
292,-2.148441,-79.864986,1


In [175]:
vl_df['n_visits'].value_counts()

1      22
2      16
3      13
6      13
9       8
       ..
222     1
219     1
206     1
204     1
128     1
Name: n_visits, Length: 153, dtype: int64

In [176]:
vl_df_2 = vl_df.loc[vl_df['n_visits'] > 1]

In [177]:
vl_df_2['n_visits'].value_counts()

2      16
3      13
6      13
9       8
8       7
       ..
239     1
222     1
219     1
206     1
128     1
Name: n_visits, Length: 152, dtype: int64

In [178]:
tdf_n_visits = tdf_mobility.merge(vl_df_2, on=['lat', 'lng'])
tdf_n_visits

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid,distance_straight_line,number_of_locations,n_visits
0,2019-02-21,09:10:48,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3,2
1,2019-02-07,09:12:07,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3,2
2,2019-04-16,21:30:15,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3,407
3,2019-04-09,20:35:36,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3,407
4,2019-03-24,18:07:12,Juan Montalvo,92168,5.0,2.0,263.0,-2.124883,-79.927019,%..))+.&.&,0.694118,2,407
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70405,2019-12-11,15:41:04,Garzota,95337,5.0,2.0,367.0,-2.146365,-79.889863,%.-*&.*(&+,0.643142,2,22
70406,2019-12-08,13:20:24,Garzota,95337,5.0,15.0,367.0,-2.146365,-79.889863,%.-*&.*(&+,0.643142,2,22
70407,2019-12-13,12:00:36,Garzota,95337,5.0,15.0,367.0,-2.146365,-79.889863,%.-*&.*(&+,0.643142,2,22
70408,2019-12-16,12:49:23,Garzota,95337,5.0,15.0,367.0,-2.146365,-79.889863,%.-*&.*(&+,0.643142,2,22


In [179]:
tdf_n_visits['distance_straight_line'].isna().sum()

0

In [180]:
hl_df = home_location(tdf_n_visits)
hl_df.head()

100%|██████████| 4456/4456 [00:07<00:00, 621.30it/s]


,uid,lat,lng
0,%%%!!&+%*+.-,-2.090521,-79.941138
1,%%%!!&.(()*),-2.150990,-79.930209
2,%%%!!'%.%!-(,-2.125813,-79.922279
3,%%%!!'&())++,-2.121863,-79.922679
4,%%%!!)%!*-+*,-2.278056,-79.883938


In [181]:
hl_df.isna().sum()

uid    0
lat    0
lng    0
dtype: int64

In [182]:
hl_df.rename(columns={'lat': 'home_lat', 'lng': 'home_lng'}, inplace=True)

In [183]:
tdf_info_all_merge = tdf_n_visits.merge(hl_df, on='uid')
tdf_info_all_merge

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid,distance_straight_line,number_of_locations,n_visits,home_lat,home_lng
0,2019-02-21,09:10:48,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3,2,-2.119984,-79.904241
1,2019-02-07,09:12:07,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3,2,-2.119984,-79.904241
2,2019-04-16,21:30:15,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3,407,-2.119984,-79.904241
3,2019-04-09,20:35:36,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3,407,-2.119984,-79.904241
4,2019-04-22,17:28:48,Samanes,92593,6.0,2.0,367.0,-2.119984,-79.904241,%.+%+-')%!,4.754176,3,313,-2.119984,-79.904241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70405,2019-11-20,15:55:14,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561
70406,2019-11-12,18:35:25,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561
70407,2019-12-30,20:15:41,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561
70408,2019-12-30,20:22:23,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561


In [184]:
lf_df = location_frequency(tdf_info_all_merge, normalize=False).reset_index()

100%|██████████| 4456/4456 [00:06<00:00, 741.45it/s] 


In [185]:
lf_df

,uid,lat,lng,location_frequency
0,%%%!!&+%*+.-,-2.090521,-79.941138,8
1,%%%!!&+%*+.-,-2.090531,-79.941177,1
2,%%%!!&.(()*),-2.150990,-79.930209,5
3,%%%!!&.(()*),-2.152097,-79.930936,2
4,%%%!!'%.%!-(,-2.125813,-79.922279,23
...,...,...,...,...
10742,cVc,-2.226953,-79.906268,1
10743,cVc,-2.176709,-79.941713,1
10744,cVc,-2.126254,-79.913790,1
10745,cVc,-2.108618,-79.955168,1


In [186]:
tdf_info_all_merge = tdf_info_all_merge.merge(lf_df, on=['uid', 'lat', 'lng'])
tdf_info_all_merge

,datetime,hora,Name,comercio,proveedor,producto,ejecutivo,lat,lng,uid,distance_straight_line,number_of_locations,n_visits,home_lat,home_lng,location_frequency
0,2019-02-21,09:10:48,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3,2,-2.119984,-79.904241,2
1,2019-02-07,09:12:07,Rosales,90093,6.0,2.0,367.0,-2.115323,-79.910043,%.+%+-')%!,4.754176,3,2,-2.119984,-79.904241,2
2,2019-04-16,21:30:15,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3,407,-2.119984,-79.904241,2
3,2019-04-09,20:35:36,Juan Montalvo,92168,6.0,2.0,263.0,-2.124883,-79.927019,%.+%+-')%!,4.754176,3,407,-2.119984,-79.904241,2
4,2019-04-22,17:28:48,Samanes,92593,6.0,2.0,367.0,-2.119984,-79.904241,%.+%+-')%!,4.754176,3,313,-2.119984,-79.904241,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70405,2019-11-20,15:55:14,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561,13
70406,2019-11-12,18:35:25,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561,13
70407,2019-12-30,20:15:41,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561,13
70408,2019-12-30,20:22:23,Estero Salado,95378,5.0,15.0,373.0,-2.202591,-79.942561,%.-%.!*')',4.115365,2,71,-2.202591,-79.942561,13


In [187]:
df_info_all_merge = tdf_info_all_merge.rename(columns={'lat': 'latitud', 'lng': 'longitud', 'uid': 'cliente_final'})

In [188]:
df_info_all_merge.shape

(70410, 16)

In [189]:
df_info_all_merge.to_csv('../../data/output/df_info_all_merge.csv', index=False)